In [17]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import holidays


In [92]:
train_X = pd.read_excel("train_X.xlsx", sheet_name = None, engine = "openpyxl")
train_y = pd.read_excel( "train_y.xlsx", sheet_name = None, engine = "openpyxl")
valid_X = pd.read_excel( "valid_X.xlsx", sheet_name = None, engine = "openpyxl")
valid_y = pd.read_excel( "valid_y.xlsx", sheet_name = None, engine = "openpyxl")

In [93]:
train_mkt_data = train_X['市場データ_final'].loc[6:]
train_mkt_data.columns = list(train_X['市場データ_final'].iloc[5])
train_mkt_data =train_mkt_data.astype({'Dates': 'datetime64[ns]'})
train_mkt_data.head()

,Dates,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,HIIndex,MXEFIndex,...,TRYCurncy,BRLCurncy,COComdty,GCComdty,SIComdty,HGComdty,LBComdty,BDIYindex,XBTCurncy,VIXIndex
6,1999-04-01,0.007346,-0.001362,0.002121,0.036825,0.001509,0.009022,-0.000526,0.02157,0.001105,...,0.373,1.725,-0.034121,0.000355,0.01167,0.001594,-0.010963,891,NaN,22.06
7,1999-04-02,0,0.00573,0.003438,0.002449,-0.001432,0,0,0,0.010683,...,0.3754,1.725,0,0,0,0,0,891,NaN,NaN
8,1999-04-05,0.023604,0.000543,0.009767,0.000611,-0.001207,0,0,0,0.002008,...,0.3761,1.7375,0,-0.006028,-0.021877,0.002387,-0.017376,891,NaN,22.19
9,1999-04-06,-0.00225,0.004067,0.00154,0.004274,-0.002569,0.025426,0.010093,0,0.009576,...,0.376,1.7325,0.014266,0.005351,0.0061,-0.003968,-0.01372,883,NaN,22.65
10,1999-04-07,0.006013,-0.00081,0.001919,0.00304,0.006212,-0.008174,-0.000521,0.050764,0.010974,...,0.3761,1.7325,-0.038848,0.001774,0.002425,0.030279,0.004328,873,NaN,22.81


In [94]:
train_ind_data = train_X['経済指標データ_final']
train_ind_col = list(train_ind_data.iloc[5])
train_ind_data =train_ind_data.loc[6:]
train_ind_data.columns = train_ind_col
train_ind_data =train_ind_data.astype({'Dates': 'datetime64[ns]'})
train_ind_data.head()

,Dates,NAPMPMIIndex,NAPMNEWOIndex,NAPMPRODIndex,NAPMSUPLIndex,NAPMEMPLIndex,NAPMINVIndex,NAPMBACKIndex,NAPMEXPTIndex,NAPMIMPTIndex,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
6,1997-03-31,53.1,57.6,58.9,52.1,45.9,41.3,48.5,54.6,53.7,...,NaN,1486,1445,NaN,801,21.1,NaN,57.5,118.5,100
7,1997-04-01,55,58.6,55,53.1,50.8,43.5,51.5,55.7,48.3,...,NaN,1486,1445,NaN,801,21.1,NaN,57.5,118.5,100
8,1997-04-02,55,58.6,55,53.1,50.8,43.5,51.5,55.7,48.3,...,NaN,1486,1445,NaN,801,21.1,NaN,57.5,118.5,100
9,1997-04-03,55,58.6,55,53.1,50.8,43.5,51.5,55.7,48.3,...,NaN,1486,1445,NaN,801,21.1,NaN,57.5,118.5,100
10,1997-04-04,55,58.6,55,53.1,50.8,43.5,51.5,55.7,48.3,...,NaN,1486,1445,NaN,801,21.1,NaN,57.5,118.5,100


In [95]:
train_df = train_y['Sheet1']
train_df = pd.merge(train_df, train_mkt_data, on='Dates')
train_df = pd.merge(train_df, train_ind_data, on='Dates')
train_df.head(5)

,Dates,dec_rate,signal,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,1999-04-01,-0.004176,0,0.007346,-0.001362,0.002121,0.036825,0.001509,0.009022,-0.000526,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1999-04-02,-0.002029,0,0,0.00573,0.003438,0.002449,-0.001432,0,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,1999-04-05,-0.006016,0,0.023604,0.000543,0.009767,0.000611,-0.001207,0,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,1999-04-06,-0.008308,0,-0.00225,0.004067,0.00154,0.004274,-0.002569,0.025426,0.010093,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,1999-04-07,0.000000,0,0.006013,-0.00081,0.001919,0.00304,0.006212,-0.008174,-0.000521,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7


In [96]:
valid_mkt_data = valid_X['市場データ_final'].loc[6:]
valid_mkt_data.columns = list(valid_X['市場データ_final'].iloc[5])
valid_mkt_data =valid_mkt_data.astype({'Dates': 'datetime64[ns]'})

valid_ind_data = valid_X['経済指標データ_final']
valid_ind_col = list(valid_ind_data.iloc[5])
valid_ind_data =valid_ind_data.loc[6:]
valid_ind_data.columns = valid_ind_col
valid_ind_data =valid_ind_data.astype({'Dates': 'datetime64[ns]'})

valid_df = valid_y['Sheet1']
valid_df = pd.merge(valid_df, valid_mkt_data, on='Dates')
valid_df = pd.merge(valid_df, valid_ind_data, on='Dates')
valid_df.head(5)

,Dates,dec_rate,signal,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,2017-01-02,-0.001239,0,0,0,0,0,0,0.005493,0.001949,...,54,1090,1201,5.61,592,21.5,9,54.6,113.7,98.2
1,2017-01-03,-0.000656,0,0.007267,-0.001383,0.002077,0,0,0.003338,-0.007661,...,54,1090,1201,5.61,592,21.5,9,54.6,113.7,98.2
2,2017-01-04,0.000000,0,0.005216,0,0.002087,0.027749,-0.001331,-0.000605,-0.000123,...,54,1090,1201,5.61,592,21.5,9,54.6,113.7,98.2
3,2017-01-05,0.000000,0,0,0.005791,0.003475,-0.006623,0.0008,0,0.002145,...,50,1090,1201,5.61,592,21.5,9,54.6,113.7,98.2
4,2017-01-06,0.000000,0,0.003202,-0.00338,-0.000747,-0.005128,0,0.002724,-0.00428,...,50,1090,1201,5.61,592,21.5,9,54.6,113.7,98.2


In [97]:
df=pd.concat([train_df,valid_df])
df.head()

,Dates,dec_rate,signal,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,1999-04-01,-0.004176,0,0.007346,-0.001362,0.002121,0.036825,0.001509,0.009022,-0.000526,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1999-04-02,-0.002029,0,0,0.00573,0.003438,0.002449,-0.001432,0,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,1999-04-05,-0.006016,0,0.023604,0.000543,0.009767,0.000611,-0.001207,0,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,1999-04-06,-0.008308,0,-0.00225,0.004067,0.00154,0.004274,-0.002569,0.025426,0.010093,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,1999-04-07,0.000000,0,0.006013,-0.00081,0.001919,0.00304,0.006212,-0.008174,-0.000521,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7


In [99]:
df.shape

(6127, 136)

In [100]:
#金利3年を削除リストに
eliminate_list = ["USGG3YRIndex","USYC2Y3YIndex","USYC3Y5YIndex","USYC3Y7YIndex","USYC3Y10Index","USYC3Y20Index","USYC3Y30Index","BF020305Index","BF020307Index","BF020310Index","BF020320Index","BF020330Index","BF030507Index","BF030510Index","BF030520Index","BF030530Index","BF030710Index","BF030720Index","BF030730Index","BF031020Index","BF031030Index"]
#ぜんぶNanの列を削除リストに
for i in range(len(train_df.columns)):
    if train_df.isnull().all()[i] == True:
        eliminate_list.append(train_df.columns[i])
eliminate_list

['USGG3YRIndex',
 'USYC2Y3YIndex',
 'USYC3Y5YIndex',
 'USYC3Y7YIndex',
 'USYC3Y10Index',
 'USYC3Y20Index',
 'USYC3Y30Index',
 'BF020305Index',
 'BF020307Index',
 'BF020310Index',
 'BF020320Index',
 'BF020330Index',
 'BF030507Index',
 'BF030510Index',
 'BF030520Index',
 'BF030530Index',
 'BF030710Index',
 'BF030720Index',
 'BF030730Index',
 'BF031020Index',
 'BF031030Index',
 'XLCEquity',
 'USGG20YRIndex',
 'USYC2Y20Index',
 'USYC3Y20Index',
 'USYC5Y20Index',
 'USYC7Y20Index',
 'USYC1020Index',
 'USYC2030Index',
 'BF020320Index',
 'BF020520Index',
 'BF020720Index',
 'BF021020Index',
 'BF030520Index',
 'BF030720Index',
 'BF031020Index',
 'BF050720Index',
 'BF051020Index',
 'BF052030Index',
 'BF071020Index',
 'BF072030Index',
 'BF102030Index']

In [111]:
df = df.drop(eliminate_list, axis=1)
df.shape

KeyError: "['USGG3YRIndex', 'USYC2Y3YIndex', 'USYC3Y5YIndex', 'USYC3Y7YIndex', 'USYC3Y10Index', 'USYC3Y20Index', 'USYC3Y30Index', 'BF020305Index', 'BF020307Index', 'BF020310Index', 'BF020320Index', 'BF020330Index', 'BF030507Index', 'BF030510Index', 'BF030520Index', 'BF030530Index', 'BF030710Index', 'BF030720Index', 'BF030730Index', 'BF031020Index', 'BF031030Index', 'XLCEquity', 'USGG20YRIndex', 'USYC2Y20Index', 'USYC3Y20Index', 'USYC5Y20Index', 'USYC7Y20Index', 'USYC1020Index', 'USYC2030Index', 'BF020320Index', 'BF020520Index', 'BF020720Index', 'BF021020Index', 'BF030520Index', 'BF030720Index', 'BF031020Index', 'BF050720Index', 'BF051020Index', 'BF052030Index', 'BF071020Index', 'BF072030Index', 'BF102030Index'] not found in axis"

In [110]:
df.head()

,Dates,dec_rate,signal,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,1999-04-01,-0.004176,0,0.007346,-0.001362,0.002121,0.036825,0.001509,0.009022,-0.000526,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1999-04-02,-0.002029,0,0,0.00573,0.003438,0.002449,-0.001432,0,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,1999-04-05,-0.006016,0,0.023604,0.000543,0.009767,0.000611,-0.001207,0,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,1999-04-06,-0.008308,0,-0.00225,0.004067,0.00154,0.004274,-0.002569,0.025426,0.010093,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,1999-04-07,0.000000,0,0.006013,-0.00081,0.001919,0.00304,0.006212,-0.008174,-0.000521,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7


In [90]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
usb = CustomBusinessDay(calendar = USFederalHolidayCalendar())
us_holiday = pd.date_range('1/4/1999', '24/9/2022', freq=usb)

C:\Users\aaa\AppData\Local\Temp\ipykernel_6592\1331842619.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  us_holiday = pd.date_range('1/4/1999', '24/9/2022', freq=usb)


各日付の休日情報を追加する

In [115]:
df['Dates']=df['Dates'].astype('datetime64[ns]')#if you dont change the form of this col into datetime64, it wont work!!!
df.insert(0,'Index2',df.index)
df.insert(2, 'Year', df['Dates'].dt.year)
df.insert(3, 'Month', df['Dates'].dt.month)
df.insert(4, 'Day', df['Dates'].dt.day)
df.insert(5, 'DayOfWeek', df['Dates'].dt.dayofweek)
df.insert(6, 'AgeInDays',np.nan)
df.insert(7, 'Holiday',np.nan)

In [117]:
df

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,NaN,NaN,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1,1999-04-02,1999,4,2,4,NaN,NaN,-0.002029,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,2,1999-04-05,1999,4,5,0,NaN,NaN,-0.006016,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,3,1999-04-06,1999,4,6,1,NaN,NaN,-0.008308,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,4,1999-04-07,1999,4,7,2,NaN,NaN,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,1490,2022-09-19,2022,9,19,0,NaN,NaN,-0.027976,1,...,48.2,1446,1674,4.81,511,-9.9,-1.5,52.2,103.2,58.2
1491,1491,2022-09-20,2022,9,20,1,NaN,NaN,-0.013651,1,...,48.2,1575,1517,4.81,511,-9.9,-1.5,52.2,103.2,58.2
1492,1492,2022-09-21,2022,9,21,2,NaN,NaN,-0.020086,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2
1493,1493,2022-09-22,2022,9,22,3,NaN,NaN,-0.024371,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2


In [118]:
for i in range(1,df.shape[0]):
    df.iloc[i,6] = df.iloc[i,0] - df.iloc[0,0]
    df.iloc[i,7] = int(df.iloc[i,0] in us_holiday)

df.iloc[0,6]= dt.timedelta(days=0)
df.iloc[0,7]= int(df.iloc[0,7] not in us_holiday)



df = df.reset_index()
df = df.drop('index', axis=1)
df.head()

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0:00:00,1.0,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1.0,0.0,-0.002029,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2.0,0.0,-0.006016,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3.0,0.0,-0.008308,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4.0,0.0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7


In [119]:
df.loc[0,'AgeInDays'] = 0
df = df.astype({'AgeInDays': 'float'})
df = df.astype({'AgeInDays': 'int'})
df = df.astype({'Holiday': 'int'})
df

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,-0.002029,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,-0.006016,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,-0.008308,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,1490,2022-09-19,2022,9,19,0,1490,0,-0.027976,1,...,48.2,1446,1674,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6123,1491,2022-09-20,2022,9,20,1,1491,0,-0.013651,1,...,48.2,1575,1517,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6124,1492,2022-09-21,2022,9,21,2,1492,0,-0.020086,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2
6125,1493,2022-09-22,2022,9,22,3,1493,0,-0.024371,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2


In [120]:
df3 = df
df4 = df
df5 = df

In [121]:
df.shape[0]

6127

ある日付に対して、この日の五日後のdec_rate値をこちらの行に移動する

In [122]:
for i in range(1, df.shape[0]):
    try:
        df3.iloc[i,8] = df3.iloc[i+5, 8]
    except:
        df3.iloc[i, 8] = np.nan

In [123]:
df3.head(15)

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
5,5,1999-04-08,1999,4,8,3,5,0,-0.010289,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
6,6,1999-04-09,1999,4,9,4,6,0,-0.015361,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
7,7,1999-04-12,1999,4,12,0,7,0,-0.024822,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
8,8,1999-04-13,1999,4,13,1,8,0,-0.020046,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
9,9,1999-04-14,1999,4,14,2,9,0,-0.012807,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7


In [124]:
df4 = df.fillna(method='ffill')
df4.head(15)

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
5,5,1999-04-08,1999,4,8,3,5,0,-0.010289,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
6,6,1999-04-09,1999,4,9,4,6,0,-0.015361,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
7,7,1999-04-12,1999,4,12,0,7,0,-0.024822,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
8,8,1999-04-13,1999,4,13,1,8,0,-0.020046,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
9,9,1999-04-14,1999,4,14,2,9,0,-0.012807,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7


In [125]:
df5 = df3.fillna(method='ffill')
df5.head(15)

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
5,5,1999-04-08,1999,4,8,3,5,0,-0.010289,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
6,6,1999-04-09,1999,4,9,4,6,0,-0.015361,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
7,7,1999-04-12,1999,4,12,0,7,0,-0.024822,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
8,8,1999-04-13,1999,4,13,1,8,0,-0.020046,1,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7
9,9,1999-04-14,1999,4,14,2,9,0,-0.012807,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57.0,133.9,105.7


データの定常化、まずは日付等を取り出し、そして数値の部分にlambdaｘを用いてすべての値が定常化した。最後日付と定常値を合併する

In [126]:
df5

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.10,881,10.4,NaN,57.0,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,52.7,1799,1720,5.10,881,10.4,NaN,57.0,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,55.3,1799,1720,5.10,881,10.4,NaN,57.0,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,55.3,1799,1720,5.10,881,10.4,NaN,57.0,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.10,881,10.4,NaN,57.0,133.9,105.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,1490,2022-09-19,2022,9,19,0,1490,0,-0.030041,1,...,48.2,1446,1674,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6123,1491,2022-09-20,2022,9,20,1,1491,0,-0.030041,1,...,48.2,1575,1517,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6124,1492,2022-09-21,2022,9,21,2,1492,0,-0.030041,1,...,48.2,1575,1517,4.80,511,-9.9,-1.5,52.2,103.2,58.2
6125,1493,2022-09-22,2022,9,22,3,1493,0,-0.030041,1,...,48.2,1575,1517,4.80,511,-9.9,-1.5,52.2,103.2,58.2


In [127]:
df5.to_csv('train&valid_yX_after_process.csv')

In [128]:
train_df=df5[:]
dfl=train_df.iloc[:,:10]
dfl.head(10)

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0
5,5,1999-04-08,1999,4,8,3,5,0,-0.010289,0
6,6,1999-04-09,1999,4,9,4,6,0,-0.015361,0
7,7,1999-04-12,1999,4,12,0,7,0,-0.024822,1
8,8,1999-04-13,1999,4,13,1,8,0,-0.020046,1
9,9,1999-04-14,1999,4,14,2,9,0,-0.012807,0


In [129]:
dfr=train_df.iloc[:,10:]
dfr=dfr.apply(lambda x: (x-x.mean())/ x.std(), axis=0)#定常化操作
dfr.head()

,ESIndex,TYComdty,TargetPortDailyRtn,NKIndex,JBComdty,VGIndex,RXComdty,HIIndex,MXEFIndex,LUACTRUUIndex,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0.583601,-0.403522,0.413591,2.487495,0.745984,0.604695,-0.182523,1.412765,0.075168,-1.472374,...,0.073469,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
1,-0.019343,1.538316,0.691536,0.149935,-0.776941,-0.011737,-0.031755,-0.017186,0.898052,-1.472374,...,0.073469,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
2,1.918061,0.118057,2.027243,0.024910,-0.660779,-0.011737,-0.031755,-0.017186,0.152700,-1.464296,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
3,-0.204000,1.083094,0.291097,0.273968,-1.365737,1.725569,2.859499,-0.017186,0.802987,-1.458503,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
4,0.474184,-0.252320,0.370987,0.190057,3.180717,-0.570274,-0.181095,3.348191,0.923072,-1.458676,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494


In [130]:
dfr.mean()

ESIndex               1.855506e-17
TYComdty             -1.971475e-17
TargetPortDailyRtn    1.159691e-17
NKIndex              -1.739537e-17
JBComdty              4.638764e-18
                          ...     
OUTFGAFIndex          0.000000e+00
EMPRGBCIIndex        -3.083730e-17
CHPMINDXIndex         2.783259e-16
CONCCONFIndex         2.041056e-16
CONSSENTIndex         0.000000e+00
Length: 96, dtype: float64

In [131]:
dfr.std()

ESIndex               1.0
TYComdty              1.0
TargetPortDailyRtn    1.0
NKIndex               1.0
JBComdty              1.0
                     ... 
OUTFGAFIndex          1.0
EMPRGBCIIndex         1.0
CHPMINDXIndex         1.0
CONCCONFIndex         1.0
CONSSENTIndex         1.0
Length: 96, dtype: float64

In [132]:
df6 = pd.concat([dfl, dfr], axis=1)
df6.head()

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,0.073469,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,0.073469,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,0.671427,1.100428,0.842937,-0.594433,0.554169,0.109676,NaN,0.195015,1.429954,1.561494


In [133]:
df7 = df
df7

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex,NHSPSTOTIndex,NHSPATOTIndex,ETSLTOTLIndex,NHSLTOTIndex,OUTFGAFIndex,EMPRGBCIIndex,CHPMINDXIndex,CONCCONFIndex,CONSSENTIndex
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,52.7,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,55.3,1799,1720,5.1,881,10.4,NaN,57,133.9,105.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,1490,2022-09-19,2022,9,19,0,1490,0,NaN,1,...,48.2,1446,1674,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6123,1491,2022-09-20,2022,9,20,1,1491,0,NaN,1,...,48.2,1575,1517,4.81,511,-9.9,-1.5,52.2,103.2,58.2
6124,1492,2022-09-21,2022,9,21,2,1492,0,NaN,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2
6125,1493,2022-09-22,2022,9,22,3,1493,0,NaN,1,...,48.2,1575,1517,4.8,511,-9.9,-1.5,52.2,103.2,58.2


In [134]:
train_ind_info = pd.read_csv('train_ind_info.csv', index_col=0)
train_mkt_info = pd.read_csv('train_mkt_info.csv', index_col=0)

各原数値の列に対して、各日付の翌日の値と当日の値の差をdiffで求め、dfの新たなコピーdf7にそれぞれのdiffを追加する

In [135]:
mkt_ind_info = pd.concat([train_mkt_info,train_ind_info]) 
originalValueList = []#原数値の特定
originalValueNumList = []#原数値の特定
for i in mkt_ind_info.index:
    if mkt_ind_info.at[i, 'データ種類'] == '原数値':
        originalValueList.append(i)
        originalValueNumList.append(df7.columns.get_loc(i))
originalValueList

['LUACTRUUIndex',
 'LUACTRJHIndex',
 'LCA3TRUUIndex',
 'LCA2TRUUIndex',
 'LCA1TRUUIndex',
 'LCB1TRUUIndex',
 'BCBATRUUIndex',
 'BCBHTRUUIndex',
 'EMUSTRUUIndex',
 'XLKEquity',
 'XLFEquity',
 'XLVEquity',
 'XLEEquity',
 'XLYEquity',
 'XLPEquity',
 'XLIEquity',
 'XLUEquity',
 'XLBEquity',
 'XLREEquity',
 'USGG2YRIndex',
 'USGG5YRIndex',
 'USGG7YRIndex',
 'USGG10YRIndex',
 'USGG30YRIndex',
 'USYC2Y5YIndex',
 'USYC2Y7YIndex',
 'USYC2Y10Index',
 'USYC2Y30Index',
 'USYC5Y7YIndex',
 'USYC5Y10Index',
 'USYC5Y30Index',
 'USYC7Y10Index',
 'USYC7Y30Index',
 'USYC1030Index',
 'BF020507Index',
 'BF020510Index',
 'BF020530Index',
 'BF020710Index',
 'BF020730Index',
 'BF021030Index',
 'BF050710Index',
 'BF050730Index',
 'BF051030Index',
 'BF071030Index',
 'DXYCurncy',
 'JPYCurncy',
 'EURCurncy',
 'GBPCurncy',
 'AUDCurncy',
 'CADCurncy',
 'CNYCurncy',
 'TRYCurncy',
 'BRLCurncy',
 'BDIYindex',
 'XBTCurncy',
 'VIXIndex',
 'NAPMPMIIndex',
 'NAPMNEWOIndex',
 'NAPMPRODIndex',
 'NAPMSUPLIndex',
 'NAPMEMPLIn

In [136]:
originalValueListDiff = []
for i in originalValueList:
    originalValueListDiff.append(i+str('_diff'))
originalValueListDiff

['LUACTRUUIndex_diff',
 'LUACTRJHIndex_diff',
 'LCA3TRUUIndex_diff',
 'LCA2TRUUIndex_diff',
 'LCA1TRUUIndex_diff',
 'LCB1TRUUIndex_diff',
 'BCBATRUUIndex_diff',
 'BCBHTRUUIndex_diff',
 'EMUSTRUUIndex_diff',
 'XLKEquity_diff',
 'XLFEquity_diff',
 'XLVEquity_diff',
 'XLEEquity_diff',
 'XLYEquity_diff',
 'XLPEquity_diff',
 'XLIEquity_diff',
 'XLUEquity_diff',
 'XLBEquity_diff',
 'XLREEquity_diff',
 'USGG2YRIndex_diff',
 'USGG5YRIndex_diff',
 'USGG7YRIndex_diff',
 'USGG10YRIndex_diff',
 'USGG30YRIndex_diff',
 'USYC2Y5YIndex_diff',
 'USYC2Y7YIndex_diff',
 'USYC2Y10Index_diff',
 'USYC2Y30Index_diff',
 'USYC5Y7YIndex_diff',
 'USYC5Y10Index_diff',
 'USYC5Y30Index_diff',
 'USYC7Y10Index_diff',
 'USYC7Y30Index_diff',
 'USYC1030Index_diff',
 'BF020507Index_diff',
 'BF020510Index_diff',
 'BF020530Index_diff',
 'BF020710Index_diff',
 'BF020730Index_diff',
 'BF021030Index_diff',
 'BF050710Index_diff',
 'BF050730Index_diff',
 'BF051030Index_diff',
 'BF071030Index_diff',
 'DXYCurncy_diff',
 'JPYCurncy

In [137]:
df7[originalValueListDiff] = df[originalValueList].diff()
df7

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex_diff,NHSPSTOTIndex_diff,NHSPATOTIndex_diff,ETSLTOTLIndex_diff,NHSLTOTIndex_diff,OUTFGAFIndex_diff,EMPRGBCIIndex_diff,CHPMINDXIndex_diff,CONCCONFIndex_diff,CONSSENTIndex_diff
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,2.6,0,0,0.0,0,0.0,NaN,0,0.0,0.0
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,1490,2022-09-19,2022,9,19,0,1490,0,NaN,1,...,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
6123,1491,2022-09-20,2022,9,20,1,1491,0,NaN,1,...,0.0,129,-157,0.0,0,0.0,0.0,0.0,0.0,0.0
6124,1492,2022-09-21,2022,9,21,2,1492,0,NaN,1,...,0.0,0,0,-0.01,0,0.0,0.0,0.0,0.0,0.0
6125,1493,2022-09-22,2022,9,22,3,1493,0,NaN,1,...,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [138]:
df

,Index2,Dates,Year,Month,Day,DayOfWeek,AgeInDays,Holiday,dec_rate,signal,...,NAPMNIMPIndex_diff,NHSPSTOTIndex_diff,NHSPATOTIndex_diff,ETSLTOTLIndex_diff,NHSLTOTIndex_diff,OUTFGAFIndex_diff,EMPRGBCIIndex_diff,CHPMINDXIndex_diff,CONCCONFIndex_diff,CONSSENTIndex_diff
0,0,1999-04-01,1999,4,1,3,0,1,-0.004176,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1999-04-02,1999,4,2,4,1,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
2,2,1999-04-05,1999,4,5,0,2,0,0.000000,0,...,2.6,0,0,0.0,0,0.0,NaN,0,0.0,0.0
3,3,1999-04-06,1999,4,6,1,3,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
4,4,1999-04-07,1999,4,7,2,4,0,0.000000,0,...,0.0,0,0,0.0,0,0.0,NaN,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,1490,2022-09-19,2022,9,19,0,1490,0,NaN,1,...,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
6123,1491,2022-09-20,2022,9,20,1,1491,0,NaN,1,...,0.0,129,-157,0.0,0,0.0,0.0,0.0,0.0,0.0
6124,1492,2022-09-21,2022,9,21,2,1492,0,NaN,1,...,0.0,0,0,-0.01,0,0.0,0.0,0.0,0.0,0.0
6125,1493,2022-09-22,2022,9,22,3,1493,0,NaN,1,...,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0


dfの保存、df7はtrainとvalid両方有り、値のdiffもある。df５はdf7に比べてdiffが無い
生データに比べて変更点：
１、要らない列の削除
２、市場データと経済指標データが一つの表に合併
３、日付のdec_rateはその五日後のdec_rate
４、休日情報追加
５、df7の値は定常化された
６、df7の原数値部分にdiffを計算した

In [139]:
df7.to_csv('train&valid_yX_with_diff_in_original_value.csv')

In [140]:
df5.to_csv('train&valid_yX_after_process.csv')